<a href="https://colab.research.google.com/github/glasslion/machinelearninginaction-redux/blob/master/decision-tree-id3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from math import log
import operator
import numpy as np
import pandas as pd

In [0]:
def createDataSet():
    dataSet = [[1, 1, 'yes'],
           [1, 1, 'yes'],
           [1, 0, 'no'],
           [0, 1, 'no'],
           [0, 1, 'no']]
    dataSet = pd.DataFrame(dataSet, columns=['no surfacing', 'flippers', 'y'])
    dataSet
    return dataSet

In [3]:
dataSet = createDataSet()
dataSet

,no surfacing,flippers,y
0,1,1,yes
1,1,1,yes
2,1,0,no
3,0,1,no
4,0,1,no


In [0]:
X, y = dataSet.drop('y', axis=1), dataSet['y']

In [0]:
def majorityCnt(y):
    return y.value_counts().idxmax()

 ## 计算**香农熵（信息熵）**

$H=-\sum_{i=1}^{M} P_i\,log_2\,P_i$

In [0]:
def calcShannonEnt(y):
    numEntries = len(y)
    value_counts = y.value_counts()
    shannonEnt = 0.0
    for cnt in value_counts :
        prob = float(cnt)/numEntries
        shannonEnt -= prob * log(prob, 2) #log base 2
    return shannonEnt

In [7]:
print(calcShannonEnt(pd.Series([0,0,0])))
print(calcShannonEnt(pd.Series([0,0,1])))
print(calcShannonEnt(pd.Series([0,1,1])))

0.0
0.9182958340544896
0.9182958340544896


## 划分数据集



In [0]:
def splitDataSet(X, y, feat, value):
    idx = X[feat] == value
    return X[idx].drop(feat, axis=1), y[idx]

In [9]:
dataSet = createDataSet()
X, y = dataSet.drop('y', axis=1), dataSet['y']
sX, sy = splitDataSet(X, y, 'no surfacing', 0)
sX

,flippers
3,1
4,1


In [10]:
sX, sy = splitDataSet(X, y, 'no surfacing', 1)
sX

,flippers
0,1
1,1
2,0


## 寻找最佳划分

最佳划分即熵减/信息增益(information gain) 最大的划分

In [0]:
def chooseBestFeatureToSplit(X, y):
    numFeatures = len(X.columns)
    baseEntropy = calcShannonEnt(y)
    bestInfoGain = 0.0
    bestFeature = -1
    for feat in X.columns:        #iterate over all the features
        newEntropy = 0.0
        for value in X[feat].unique():
            subX, suby = splitDataSet(X, y, feat, value)
            prob = len(subX)/float(len(X))
            newEntropy += prob * calcShannonEnt(suby)
        infoGain = baseEntropy - newEntropy     #calculate the info gain; ie reduction in entropy
        if (infoGain > bestInfoGain):       #compare this to the best gain so far
            bestInfoGain = infoGain         #if better than current best, set to best
            bestFeature = feat
    return bestFeature                      #returns an integer

In [12]:
chooseBestFeatureToSplit(X, y)

'no surfacing'

In [13]:
dataSet.iloc[1:3]

,no surfacing,flippers,y
1,1,1,yes
2,1,0,no


In [14]:
chooseBestFeatureToSplit(X.iloc[1:3], y.iloc[1:3])

'flippers'

## 递归构建树

In [0]:
def createTree(X, y):
    if len(y.unique()) == 1:
        #stop splitting when all of the classes are equal
        return y.iloc[0]
    if len(X.columns) == 0: #stop splitting when there are no more features in dataSet
        return majorityCnt(y)
    bestFeat = chooseBestFeatureToSplit(X, y)
    myTree = {bestFeat:{}}
    for value in X[bestFeat].unique():
        myTree[bestFeat][value] = createTree(*splitDataSet(X, y, bestFeat, value))
    return myTree

In [16]:
tree = createTree(X, y)
tree

{'no surfacing': {0: 'no', 1: {'flippers': {0: 'no', 1: 'yes'}}}}